In [1]:
from driftdb.drift_evaluator import run_drift_evaluator, DefaultDriftEvaluator
from dotenv import load_dotenv
from github import Github
import os


load_dotenv("../../.env")
gh_token = os.getenv("GH_TOKEN")
gh_client = Github(gh_token)
repo = os.getenv("REPO") or "gh_org/repo"
commit_sha = "db23b8ac7548eca81e76d55400b8b582b7e16bf1"

drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha=commit_sha, drift_evaluator=DefaultDriftEvaluator())
drift

{'should_alert': False,
 'message': 'Drift detected:\n- 🆕 0 addition\n- ♻️ 2 modifications\n- 🗑️ 0 deletion'}

In [2]:
from driftdb.drift_evaluator import DriftEvaluatorAbstractClass
from driftdb.drift_evaluator.interface import DriftEvaluatorContext, DriftEvaluation
import pandas as pd

class MyDriftEvaluator(DriftEvaluatorAbstractClass):
    @staticmethod
    def compute_drift_evaluation(data_drift_context: DriftEvaluatorContext)-> DriftEvaluation:
        new_dataframe =  data_drift_context["after"]
        old_dataframe =  data_drift_context["before"]
        print("summary", data_drift_context["summary"])
        if data_drift_context["summary"] == None:
            return {"should_alert": False, "message": "No summary"}
        additions = data_drift_context["summary"]["added_rows"]
        deletions = data_drift_context["summary"]["deleted_rows"]
        diff = data_drift_context["summary"]["modified_patterns"]
        diff_column = diff.columns[0][0]
        diff = diff.apply(pd.to_numeric)  # Convert the values in the diff dataframe to numeric values
        
        print("computing the diff")
        monthly_booking_drift = diff[(diff_column, "other")] - diff[(diff_column, "self")]
        print("monthly_booking_drift", monthly_booking_drift)
        if(monthly_booking_drift > 1000).any():
            return {"should_alert": True, "message": "The monthly booking drift is too high"}
        return {"should_alert": False, "message": "Small drift"}
    



drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha=commit_sha, drift_evaluator=MyDriftEvaluator())
print("large_drift", drift)

driftdb.drift_evaluator.drift_evaluators_dry_run - WARNING - Drift evaluator failed: Invalid object type at position 0


summary {'added_rows': Empty DataFrame
Columns: [name, date, age]
Index: [], 'deleted_rows': Empty DataFrame
Columns: [name, date, age]
Index: [], 'modified_rows_unique_keys': Index(['['2023-01-Charlie'', ' '2023-02-Philipe']'], dtype='object'), 'modified_patterns':          unique_keys column  old_value  new_value           pattern_id
0  [2023-01-Charlie]    age         35         36  8919565425528647957
1  [2023-02-Philipe]    age         40         42 -2274900585711107816}
large_drift None


Traceback (most recent call last):
  File "pandas/_libs/lib.pyx", line 2369, in pandas._libs.lib.maybe_convert_numeric
TypeError: Invalid object type

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/driftdb/driftdb/drift_evaluator/drift_evaluators_dry_run.py", line 60, in run_drift_evaluator
    drift_evaluation = drift_evaluator.compute_drift_evaluation(data_drift_context)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9r/55k5q9b53676pnrpch95mjlc0000gn/T/ipykernel_89212/503827201.py", line 17, in compute_drift_evaluation
    diff = diff.apply(pd.to_numeric)  # Convert the values in the diff dataframe to numeric values
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/pandas/core/frame.py", line 9568, in apply
    return op.apply().__finali

In [3]:
import json
import pandas as pd 

coucou = pd.Index(['2023-01-Charlie'])